<a href="https://colab.research.google.com/github/pld000/z3/blob/main/z3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# II. Стратегии и тактики

Как солверы устроены внутри? Как они находят решения? Как научиться думать так же -- быстро и формально, как они? :)

В Z3 и подобных ему решателях воплощены тесно связанные друг с другом наборы эвристических алгоритмов доказательства, которые тщательно отбирались и настраивались вручную. Такие наборы, комбинации эвристик, очень хорошо настроены на решение ряда известных классов проблем, однако они начинают плохо работать, когда встречаются новые классы задач. Пока, по сути, солверы могут "научиться" решать новые классы задач, только когда человек-специалист хорошо "объяснит" им подходящие стратегии поиска решений.

С другой стороны, именно потому, что эти эвристики создаются и прорабатываются людьми, они достаточно легко понимаются, и их можно фактически использовать as is, отчуждённо от солверов -- при самостоятельном решении задачек.

---

*Подготовлено Высшей школой программирования Сергея Бобровского 2023*

https://vk.com/lambda_brain

Ссылки на оригинальные ноутбуки в первом занятии.


In [ ]:
!pip install "z3-solver"
from z3 import *

# 1. Тактики и цели

## 1.1. Схема работы Z3

Общая схема работы солвера Z3 такая, что он управляет группой движков логического вывода (это называется **оркестровка**), и на верхнем уровне абстракций "большие" логические шаги решения представляются как функции, называемые **тактики**. Сами тактики объединяются с помощью так называемых **тактических указаний** (tacticals), или **комбинаторов**. Тактики обрабатывают системы ограничений, которые называются **цели** (Goals).

Когда тактика применяется к некоторой цели G, возможны четыре результата.

1) Тактике удаётся показать, что G может быть достигнута (т.е., посильна);

2) Тактике удаётся показать, что G не может быть достигнута (т.е., непосильна);

3) Тактика пока не знает, посильна G или нет, и формирует набор (последовательность) подцелей;

4) Тактика завершается неудачей.

Пункт 3 напоминает очень популярный в программировании подход к проектированию "сверху вниз": декомпозируем сложную задачу на набор более простых, после чего применяем этот подход к каждой подзадаче.

Но когда мы упрощаем (редуцируем, сводим) цель G к последовательности подцелей G1, ..., Gn, мы сталкиваемся с проблемой преобразования нашей модели (системы ограничений) под требования конкретной подцели. Для этого в солверах существует внутренний преобразователь моделей.

## 1.2. Пример цели

In [ ]:
x, y = Reals('x y') # две вещественные переменные
g  = Goal() # цель
g.add(x > 0, y > 0, x == y + 1 + 1) # добавим цели набор ограничений

# цель выведется просто как набор ограничений
print(g)


Далее добавим две встроенные в Z3 тактики решения: `simplify` и `solve-eqs`.

In [ ]:
t1 = Tactic('simplify')
t2 = Tactic('solve-eqs')

**simplify** выполняет простые эквивалентные преобразования выражений (например, x+2+x+2 = 2*x+4).

**solve-eqs** -- это классический метод исключения переменных Гаусса (редукция по столбцам), который вдобавок расширен работой не только с линейной арифметикой, но и, вообще, умеет избавляться практически от любых лишних переменных.

In [ ]:
# посмотреть список всех тактик:
print(tactics())

# посмотреть краткое описание всех тактик:
# print(describe_tactics())

# посмотреть краткое описание конкретной тактики:
print(tactic_description('simplify'))
print(tactic_description('solve-eqs'))


In [ ]:
# посмотрим на промежуточные результаты
# как работают тактики (булевы формулы традиционно выводятся в инвертированном виде):
print(t1(g))

В данном случае упрощено было 2+2 = 4.

In [ ]:
print(t2(g))

А теперь последнее ограничение `x == y + 1 + 1` тактикой было удалено, потому что оно никак не влияет на другие ограничения.

Далее объединим эти две тактики в комбинатор `Then`. Он работает так, что первый параметр (тактика simplify) применяется к входной цели, а второй параметр (тактика solve-eqs) применяется к каждой подцели, сгенерированной первой тактикой. В данном случае упрощать особо нечего, поэтому и подцель будет одна.

In [ ]:
t = Then(t1, t2)
print(t(g))

Переменная x полностью исключена из системы ограничений, потому что для любого значения y, которое положительно, всегда найдётся значение x, большее его на 2 -- переменная x никаких дополнительных ограничений не добавляет.


## 1.3. Клаузы

В Z3 называется **клаузой** любое ограничение в форме Or(f_1, ..., f_n). Тактика `split-clause` получает клаузу Or(f_1, ..., f_n) в качестве входной цели, и разбивает её на n подцелей (по одной на каждую субформулу f_i).

In [ ]:
x, y, z = Reals('x y z')
g = Goal()
g.add(Or(x == 0, x == 1),
      Or(y == 0, y == 1),
      Or(z == 0, z == 1),
      x + y + z > 2)
t = Tactic('split-clause')
r = t(g) # берём первую клаузу цели и разбиваем её на соответствующие варианты
for g in r:
    print(g)

Мы получили две клаузы -- первая логическая связка Or была разбита на два варианта. Соответственно, можно продолжать применять данную тактику к полученным подцелям.

In [ ]:
for g in r:
    print(t(g))

От всех клауз на этом уровне декомпозиции мы избавились, и далее можно применять другие тактики, подходящие для решения данной системы ограничений.

## 1.4. Комбинаторы тактик

В Z3 доступны следующие комбинаторы тактик (тактические указания):

* `Then(t, s)` применяет тактику t к входной цели, и s к каждой подцели результата t;

* `OrElse(t, s)` применяет тактику t к входной цели, и если она заканчивается неудачей, возвращает результат применения тактики s к входной цели;

* `Repeat(t)` применяет тактику t снова и снова, пока не останется ни одной подцели;

* `Repeat(t, n)` применяет тактику t снова и снова, пока не останется ни одной подцели, но количество попыток не должно превышать заданный порог n;

* `TryFor(t, ms)` применяет тактику t к входной цели, и если она не отрабатывает за ms миллисекунд, возвращается неудача;

* `With(t, params)` применяет тактику t с заданными параметрами params.

In [ ]:
# пример
x, y, z = Reals('x y z')
g = Goal()
g.add(Or(x == 0, x == 1),
      Or(y == 0, y == 1),
      Or(z == 0, z == 1),
      x + y + z > 2)

# один шаг разбиения на клаузы:
t = Tactic('split-clause')
print(t(g))


Видно, что была разобрана только одна клауза, а остальные остались неразобранными. Чтобы перевести все клаузы в разобранный вид, воспользуемся комбинатором `Repeat`.

In [ ]:
split_all = Repeat(Tactic('split-clause')) # вылетит с исключением
print(split_all(g))

Теперь возникнет исключение, потому что, когда все клаузы будут разобраны, `split-clause` завершится неудачей (нету значений, с которыми она могла бы работать).

На такой случай надо предусмотреть стандартный вариант: к итоговому результату, когда основная тактика отработает и завершится неудачей, применить тактику `skip`, которая ничего не делает (просто возвращает входную цель). Для этого воспользуемся комбинатором `OrElse`.

In [ ]:
split_all = Repeat(OrElse(Tactic('split-clause'),
                          Tactic('skip')))
print(split_all(g))

for s in split_all(g): # или перебрать подцели по одной
  print(s)

In [ ]:
# можно ограничиться одной итерацией, тогда не все клаузы будут разобраны:
split_at_most_2 = Repeat(OrElse(Tactic('split-clause'),
                          Tactic('skip')),
                         1) # ограничение на количество проб основной тактики
print(split_at_most_2(g))


Вернёмся к предыдущей Repeat, когда все клаузы были разобраны, и система получилась прозрачной: к ней наверняка подойдёт метод Гаусса. Соответствующую тактику solve-eqs надо применять, когда полностью отработает тактика split-clause, для этого задействуем комбинатор Then.

In [ ]:
split_solve = Then(Repeat(OrElse(Tactic('split-clause'),
                                 Tactic('skip'))),
                   Tactic('solve-eqs'))
print(split_solve(g))

В результате порождена одна (пустая) трививальная цель, которая достижима по определению.

# 2. Переводим тактики в стратегию

## 2.1. Объединяем тактики в солвер

Набор тактик (**стратегия**) может быть преобразован в конкретный солвер (например, для решения класса похожих задач) с помощью метода `solver()`. Если тактика выдаёт пустую цель, то соответствующий солвер возвращает `sat`. Если тактика выдаёт единственную цель, содержащую False, то солвер возвращает `unsat`. В остальных случаях он возвращает `unknown`.

In [ ]:
# создаём солвер на основе комбинаторов и тактик
split_solver = Then(Repeat(OrElse(Tactic('split-clause'),
                                 Tactic('skip'))),
                   Tactic('solve-eqs')).solver()

# добавляем солверу систему ограничений
split_solver.add(Or(x == 0, x == 1))
split_solver.add(Or(y == 0, y == 1))
split_solver.add(Or(z == 0, z == 1))
split_solver.add(x + y + z > 2)

# проверяем, есть ли решение?
print(split_solver.check()) # sat

m = split_solver.model() # вытаскиваем модель

# вычисляем результирующие значения нужных переменных
print(m.evaluate(x))
print(m.evaluate(y))
print(m.evaluate(z))

# или сразу так:
print(m)

## 2.2. Стратегия для битовых формул

Подберём подходящую стратегию для решения битовых выражений. В дополнение к тактикам `simplify` и `solve-eqs` дополнительно понадобятся тактики `bit-blast` (алгоритмы bit-blasting для решения сложных и больших битовых формул) и `sat` (классический пропозиционный решатель для работы с формулами, где все переменные логических типов).

Обратите внимание, что для параметров Then явно не указывается Tactic(): все комбинаторы Z3 автоматически вызывают Tactic() для параметра, если он строкового типа.

Команда `solve_using` -- вариант команды `solve`, для которой дополнительно специфицируется конкретный солвер.

In [ ]:
bv_solver = Then('simplify',
                 'solve-eqs',
                 'bit-blast',
                 'sat').solver()

x, y = BitVecs('x y', 16)

solve_using(bv_solver, x | y == 13, x > y) # покажите решение!

Рассмотрим этот процесс "изнутри", обращением к Z3 API, без solve_using.

In [ ]:
# тактика simplify с помощью комбинатора With
# дополнена параметром mul2concat (не знаю, что он значит :)
bv_solver = Then(With('simplify', mul2concat=True),
                 'solve-eqs',
                 'bit-blast',
                 'aig', # ещё одна тактика, которая сжимает логические формулы
                        # с помощью инвертированных графов
                 'sat').solver()
x, y = BitVecs('x y', 16)

# добавляем ограничение солверу:
bv_solver.add(x*32 + y == 13, x & y < 10, y > -100)

# решение есть?
print(bv_solver.check())

m = bv_solver.model()
print(m) # показать решение

# проверочные вычисления
print(x*32 + y, "==", m.evaluate(x*32 + y))
print(x & y, "==", m.evaluate(x & y))

## 2.3. Ещё примеры стратегий

Стандартный солвер Z3 доступен в виде отдельной тактики `smt`.

In [ ]:
x, y = Ints('x y')
s = Tactic('smt').solver()
s.add(x > y + 1)
print(s.check())
print(s.model())

Пример стратегии в формате солвера, которую можно применять для решения систем арифметических уравнений, когда каждая переменная ограничена и снизу, и сверху.

In [ ]:
s = Then(With('simplify', arith_lhs=True, som=True),
         'normalize-bounds', 'lia2pb', 'pb2bv',
         'bit-blast', 'sat').solver()
x, y, z = Ints('x y z')
solve_using(s,
            x > 0, x < 10,
            y > 0, y < 10,
            z > 0, z < 10,
            3*y + 2*x == z)


In [ ]:
# В случае же, если переменные ограничений не имеют, солвер откажется работать
s.reset() # очищаем солвер от текущей системы ограничений
solve_using(s, 3*y + 2*x == z) # fail

Тактики можно комбинировать с солверами. Например, мы можем взять конкретную тактику, применить её к цели, получить набор подцелей, затем выбрать одну из подцелей, и решить её с помощью солвера.

In [ ]:
# набор тактик
t = Then('simplify',
         'normalize-bounds',
         'solve-eqs')

x, y, z = Ints('x y z')
g = Goal()
g.add(x > 10, y == x + 3, z > y) # цель с ограничениями

r = t(g) # r содержит только одну подцель
print(r)

s = Solver()
s.add(r[0]) # ограничение подцели
print(s.check()) # sat

print(s.model()) # модель для подцели
# преобразовываем эту модель в модель для главной цели с помощью convert_model()
print(r[0].convert_model(s.model()))

Резюме, что мастерство использования SMT-солвера подразумевает прежде всего хорошее знание его тактик решения различных классов задач, и умение их комбинировать. Похожий подход существует и в машинном обучении на базе нейронных сетей, где требуется хорошо знать разные модели и подбирать и настраивать их под конкретную задачу.

## 2.4. Метрики формул
Метрики, меры (probes, formula measures) позволяют оценивать количественные и качественные (да/нет) характеристики различных целей.

In [ ]:
describe_probes() # список всех метрик Z3

In [ ]:
x, y, z = Reals('x y z')
g = Goal()
g.add(x + y + z > 0)

p = Probe('num-consts') # количество переменных в системе ограничений
print("num-consts:", p(g))

Комбинатор `FailIf(p)` формирует исключение, если его аргумент выдаёт sat.

Комбинатор `If(p, t1, t2)` работает как условный оператор -- выбирает тактику t1 если p истинно (sat), иначе выбирается тактика t2. If -- это сокращённая запись `OrElse(Then(FailIf(Not(p)), t1), t2)`.

Комбинатор `When(p, t)` -- это сокращённая запись `If(p, t, 'skip')`.

In [ ]:
x, y, z = Reals('x y z')
g = Goal()
g.add(x**2 - y**2 >= 0)

p = Probe('num-consts') # 2

# выберем подходящую тактику в зависимости от количества переменных
t = If(p > 2, 'simplify', 'factor')

print(t(g))

g = Goal()
g.add(x + x + y + z >= 0, x**2 - y**2 >= 0)
print(t(g))